# Задача 1

Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

# Задача 2

Определить, с каких источников и в какое время суток чаще всего заходят роботы.

### Предварительные шаги

In [1]:
# Импортируем необходимые библиотеки.
import pandas as pd
import json
import requests
from pprint import pprint

# Зададим токен и передадим его в заголовок будущих запросов.
token = 'AQAAAABEKJdNAAg6waJ9YuN1lU13sdEibkmnTZ8'
headers = {'Authorization': 'OAuth' + token}

### Решение задачи 1

Необходимо оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

Чтобы определить, является ли посетитель сайта роботом, нам потребуется группировка ***ym:s:isRobot***. Оценивать их будем по количеству визитов ***ym:s:visits*** и количеству уникальных посетителей ***ym:s:users***.

In [2]:
# Задаем параметры первого запроса.
params1 = {
    'metrics': 'ym:s:visits,ym:s:users',
    'dimensions': 'ym:s:isRobot',
    'date1': '2020-09-01',  
    'date2': '2020-09-30',     
    'ids': 30177909,
    'accuracy':'full',
    'limit':100000
}

# Отправляем запрос.
response1 = requests.get(
    'https://api-metrika.yandex.net/stat/v1/data', 
    params=params1, 
    headers=headers
)
# Если запрос корректный, код ответа будет равен 200.
print(response1.status_code)

200


In [3]:
# Запишем данные ответа в переменную metrika_data1.
metrika_data1 = response1.json()
pprint(metrika_data1)


{'contains_sensitive_data': False,
 'data': [{'dimensions': [{'id': 'no', 'name': 'People'}],
           'metrics': [3353.0, 2515.0]},
          {'dimensions': [{'id': 'yes', 'name': 'Robots'}],
           'metrics': [962.0, 794.0]}],
 'data_lag': 0,
 'max': [3353.0, 2515.0],
 'min': [962.0, 794.0],
 'query': {'adfox_event_id': '0',
           'attr_name': '',
           'attribution': 'LastSign',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2020-09-01',
           'date2': '2020-09-30',
           'dimensions': ['ym:s:isRobot'],
           'group': 'Week',
           'ids': [30177909],
           'limit': 100000,
           'metrics': ['ym:s:visits', 'ym:s:users'],
           'offline_window': '21',
           'offset': 1,
           'quantile': '50',
           'sort': ['-ym:s:visits']},
 'sample_share': 1.0,
 'sample_size': 4315,
 'sample_space': 4315,
 'sampled': False,
 'total_rows': 2,
 'total_rows_rounded': False,
 'totals': [4315.0, 3166

In [4]:
# Для более удобного представления данных создадим датафрейм.
metrika_df1 = pd.DataFrame(metrika_data1['data'])
display(metrika_df1)

,dimensions,metrics
0,"[{'name': 'People', 'id': 'no'}]","[3353.0, 2515.0]"
1,"[{'name': 'Robots', 'id': 'yes'}]","[962.0, 794.0]"


In [5]:
# Избавимся от вложенных словарей с помощью функции.
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d ={}
        for i, dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i, metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts1 = getMetrikaDataInListOfDicts(metrika_data1)
metrika_df1 = pd.DataFrame(metrika_list_of_dicts1)
display(metrika_df1)

,ym:s:isRobot,ym:s:visits,ym:s:users
0,People,3353.0,2515.0
1,Robots,962.0,794.0


In [6]:
# Переименуем столбцы.
metrika_df1.columns = ['Type of users','Visits','Users']
# Добавим столбцы, которые позволят оценить долю ботов от всего трафика.
metrika_df1['Share of all visits, %'] = metrika_df1['Visits']/metrika_df1['Visits'].sum()
metrika_df1['Share of all visits, %'] = round(metrika_df1['Share of all visits, %'], 3) * 100
metrika_df1['Share of all users, %'] = metrika_df1['Users']/metrika_df1['Users'].sum()
metrika_df1['Share of all users, %'] = round(metrika_df1['Share of all users, %'], 3) * 100
display(metrika_df1)

,Type of users,Visits,Users,"Share of all visits, %","Share of all users, %"
0,People,3353.0,2515.0,77.7,76.0
1,Robots,962.0,794.0,22.3,24.0


**Вывод:** 

В сентябре 2020 г. сайт посетили 794 робота, 24% от общего количества пользователей. Они совершили 962 визита, 22,3% от общего количества визитов.

### Решение задачи 2

Необходимо определить, с каких источников и в какое время суток чаще всего заходят роботы.

Это мы сделаем с помощью метрик ***ym:s:visits*** и ***ym:s:robotPercentage***. Сначала определим, с каких источников чаще всего заходят роботы.
В качестве группировок я выбрал модель атрибуции ***firstTrafficSource***, поскольку она лучше всего подходит, когда необходимо понять, какой из источников привлекает больше новых посетителей на сайт. Затем определим, в какое время суток чаще всего заходят роботы. Для этого подойдет группировка ***ym:s:hour***, поскольку нас интересует только время, а не даты посещений. 

In [7]:
# Задаем параметры второго запроса.
params2 = {
    'metrics': 'ym:s:visits,ym:s:robotPercentage',
    'dimensions': 'ym:s:firstTrafficSource',
    'date1': '2020-09-01',  
    'date2': '2020-09-30',     
    'ids': 30177909,
    'accuracy':'full',
    'limit':100000
}
response2 = requests.get(
    'https://api-metrika.yandex.net/stat/v1/data', 
    params=params2, 
    headers=headers
)
print(response2.status_code)

200


In [8]:
metrika_data2 = response2.json()
metrika_list_of_dicts2 = getMetrikaDataInListOfDicts(metrika_data2)
metrika_df2 = pd.DataFrame(metrika_list_of_dicts2)
metrika_df2.columns = ['Traffic source','Visits','Share of robots, %']
metrika_df2['Share of robots, %'] = round(metrika_df2['Share of robots, %'], 2)
metrika_df2['Visits of robots'] = round((metrika_df2['Visits'] * metrika_df2['Share of robots, %']) / 100, 0)
display(metrika_df2)

,Traffic source,Visits,"Share of robots, %",Visits of robots
0,Search engine traffic,3248.0,22.32,725.0
1,Direct traffic,863.0,22.60,195.0
2,Link traffic,125.0,24.00,30.0
3,Social network traffic,70.0,15.71,11.0
4,Cached page traffic,4.0,25.00,1.0
5,Internal traffic,3.0,0.00,0.0
6,Recommendation system traffic,2.0,0.00,0.0


**Вывод:**

В сентябре 2020 г. роботы чаще всего попадали на сайт через поисковые системы (725 визитов) и по прямым переходам (195 визитов). Реже они заходили по ссылкам (30 визитов), из соцсетей(11 визитов) и по кэшированным страницам (1 визит).

Как правило доля роботов от общего числа визитов составляла 22%-25%. Только при переходах из соцсетей она была заметно ниже - 15,71%.

In [9]:
# Задаем параметры третьего запроса.
params3 = {
    'metrics': 'ym:s:visits,ym:s:robotPercentage',
    'dimensions': 'ym:s:hour',
    'date1': '2020-09-01',  
    'date2': '2020-09-30',     
    'ids': 30177909,
    'accuracy':'full',
    'limit':100000
}
response3 = requests.get(
    'https://api-metrika.yandex.net/stat/v1/data', 
    params=params3, 
    headers=headers
)
print(response3.status_code)

200


In [10]:
metrika_data3 = response3.json()
metrika_list_of_dicts3 = getMetrikaDataInListOfDicts(metrika_data3)
metrika_df3 = pd.DataFrame(metrika_list_of_dicts3)
metrika_df3.columns = ['Hour','Visits','Share of robots, %']
metrika_df3['Share of robots, %'] = round(metrika_df3['Share of robots, %'], 2)
metrika_df3['Visits of robots'] = round((metrika_df3['Visits'] * metrika_df3['Share of robots, %']) / 100, 0)
display(metrika_df3.sort_values('Visits of robots', ascending=False, ignore_index=True))

,Hour,Visits,"Share of robots, %",Visits of robots
0,13:00,273.0,29.30,80.0
1,16:00,310.0,24.19,75.0
2,12:00,303.0,24.75,75.0
3,14:00,300.0,24.67,74.0
4,15:00,297.0,23.91,71.0
5,11:00,282.0,24.82,70.0
6,17:00,262.0,25.19,66.0
7,10:00,240.0,21.25,51.0
8,09:00,203.0,25.12,51.0
9,19:00,209.0,22.97,48.0


Чтобы результаты были более наглядными, сделаем визуализацию.

In [11]:
import plotly.express as px

fig = px.bar(
    metrika_df3.sort_values('Hour'),
    x='Hour',
    y='Visits of robots',
    text='Visits of robots',
    text_auto='.2s',
    title="Number of visits of robots during the day"
)
fig.update_traces(
    textfont_size=12, 
    textangle=0, 
    textposition="outside",
    cliponaxis=False
)
fig.show()

**Вывод:**

Чаще всего роботы посещали сайт в дневное время - с 11:00 до 18:00 (пик в 13:00-14:00).

Минимальная активность наблюдалась ночью и рано утром - с полуночи до 8:00.

# Итог

***Задача 1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.***

В сентябре 2020 г. сайт посетили 794 робота, 24% от общего количества пользователей. Они совершили 962 визита, 22,3% от общего количества визитов.

***Задача 2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.***

В сентябре 2020 г. роботы чаще всего попадали на сайт через поисковые системы (725 визитов) и по прямым переходам (195 визитов). Реже они заходили по ссылкам (30 визитов), из соцсетей(11 визитов) и по кэшированным страницам (1 визит). Как правило доля роботов от общего числа визитов составляла 22%-25%. Только при переходах из соцсетей она была заметно ниже - 15,71%.

Чаще всего роботы посещали сайт в дневное время - с 11:00 до 18:00 (пик в 13:00-14:00). Минимальная активность наблюдалась ночью и рано утром - с полуночи до 8:00.
